In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.cpu)

def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])

    j = tf.index(0, [m])

    def loop_body(i):
        norm2 = tf.zeros([1], tf.float32)
        def loop_body1(it):
            norm2.set(norm2 + A[it, i] ** 2)
        tf.loop(loop_body1, 0, m, 1)
        R[i, i] = tf.sqrt(norm2)
        Q[j, i] = A[j, i] / R[i, i]
        
        t, = tf.index_grid([i+1], [n])
        dotprod = tf.zeros(t.shape, tf.float32)
        def loop_body2(it):
            dotprod.set(dotprod + Q[it, i] * A[it, t])
        tf.loop(loop_body2, 0, m, 1)
        R[i, t] = dotprod
        
        p, k = tf.index_grid([0, i+1], [m, n])
        A[p, k] -= Q[p, i] * R[i, k]

        #p, k = tf.index_grid([0, i+1], [m, n])
        #R[i, t] = (Q[p, i] * A[p, k]).sum(axis=0) #TODO: implement sum reduction

    tf.loop(loop_body, 0, n-1, 1)

    norm2 = tf.zeros([1], tf.float32)
    def loop_body1(it):
        norm2.set(norm2 + A[it, n-1] ** 2)
    tf.loop(loop_body1, 0, m, 1)
    R[n-1, n-1] = tf.sqrt(norm2)
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)


TensorFrost module loaded!
QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Lines of generated code: 578



In [2]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 2.2890463418088597e-16
Error using TensorFrost: 1.1079049968100657e-07
Q:
 [[ 0.04231869  0.7240883  -0.1944231   0.4811696   0.45230588]
 [ 0.04542494  0.16168882  0.7123188   0.48781407 -0.47584942]
 [ 0.18473203  0.09451526  0.65451086 -0.4388624   0.5796173 ]
 [ 0.6026439   0.48707867 -0.1440797  -0.4334347  -0.43697852]
 [ 0.7738476  -0.45097017 -0.07522098  0.3873594   0.20513466]]
R:
 [[0.739292   0.9023142  0.6794356  0.8980583  0.73728895]
 [0.         0.7301071  0.5396791  0.6316689  0.19520906]
 [0.         0.         0.67682004 0.7367094  1.0479785 ]
 [0.         0.         0.         0.5000831  0.3639637 ]
 [0.         0.         0.         0.         0.48815954]]


In [3]:
#performance test
import time
A = np.random.rand(500, 500).astype(np.float32)

#naive NumPy QR decomposition
start = time.time()
Q, R = modified_gram_schmidt(A)
print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
print("Time for TensorFrost QR decomposition:", time.time() - start)

#householder QR decomposition
start = time.time()
Q, R = qr_decomposition(A)
print("Time for householder QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf.numpy, Rtf.numpy)))

Time for naive NumPy QR decomposition: 0.3596465587615967
Time for TensorFrost QR decomposition: 0.05354642868041992
Time for householder QR decomposition: 2.49704909324646
Time for built-in NumPy QR decomposition: 0.06352472305297852
Error: 9.28545e-05
Error using TensorFrost: 0.000101133475
